In [12]:
import excelHelper
import importlib, pandas, sys, re, os
#importlib.reload(excelHelper)
#helper = excelHelper.excelHelper()

gtf = pandas.read_csv('/opt/lib/worm_txpt_id_gene_name_map.txt', sep='\t', index_col=False)
#print(gtf)
from_gene_name = dict(zip(gtf['Gene name'].tolist(), gtf['Gene stable ID'].tolist()))
from_locus = dict(zip(gtf['Transcript stable ID'].tolist(), gtf['Gene stable ID'].tolist()))
from_simple_locus = dict(zip(
    [excelHelper.excelHelper.simplify_locus_id(x) for x in from_locus.keys()],
    from_locus.values()))


In [49]:
def try_to_find_wbid(name):
    if name in from_gene_name:
        return from_gene_name[name]
    elif name in from_locus:
        return from_locus[name]
    elif name in from_simple_locus:
        return from_simple_locus[name]
    else:
        print("Couldn't find {0}".format(name))
        #input()
        
        return ''
    
def top_500_genes(df):
    # Dataframe already sorted.
    
    genes_obs = set()
    gene_rank = []
    for gene in df['Gene name']:
        if len(gene):
            genes_obs.add(gene)
        gene_rank.append(len(genes_obs))

    df['Gene rank'] = gene_rank
    top = df[df['Gene rank']<501].copy()
    
    print('Ranked {0} genes.'.format(len(genes_obs)))
    print('Check: {0} gene names simplified to {1}.'.format(
        len(set(df['Gene name'].tolist())),len(genes_obs)))
    print('top 500 -> indexes {0} (may be >1 per gene), genes {1}'.format(
        len(top.index), len(set(top['Gene name']))))
    return top

table_dir = '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/tables/'

def add_wbid_to_file(in_file, out_file, top_500=False):
    
    xls = pandas.ExcelFile(in_file)

    writerOut = pandas.ExcelWriter(out_file)

    print(xls.sheet_names)

    df_by_sheetname = {}
    for sheetname in xls.sheet_names:
        df = pandas.read_excel(in_file, sheetname=sheetname, index=False)
        df['WBID'] = [try_to_find_wbid(x) for x in df['Gene name'].tolist()]
        #helper.add_gene_description(df)
        #del df['transcript_id']



        if top_500:
            df = top_500_genes(df)
            
        print(df.head(1))
        df_by_sheetname[sheetname] = df.copy()    
        
        df.to_excel(
            writerOut,
            sheet_name=sheetname,
            index=False,
        )
        
    writerOut.save()
###########
# Blocks

add_wbid_to_file(table_dir + 'File S6 Blocks.xlsx', table_dir + 'GO analysis File S6 Blocks.xlsx')
add_wbid_to_file(table_dir + 'Table S1 Peaks.xls', table_dir + 'GO analysis File S1 Peaks.xlsx', top_500=True)


['Block I', 'Block II', 'Block III']
  Gene name                  Description            WBID
0   enu-3.5  ENhancer of Uncoordination   WBGene00012552
Couldn't find vars-2
Couldn't find bcl-1
Couldn't find Y10G11A.90
  Gene name                               Description            WBID
0     fog-1  FOG-1 protein; Feminization Of Germline   WBGene00001481
  Gene name                                       Description            WBID
0     gld-1  Female germline-specific tumor suppressor gld-1   WBGene00001595
['SP FBF (25°C)', 'SP FBF-2 (25°C)', 'SP FBF-1 (25°C)', 'OO FBF-1 (20°C)', 'OO FBF-2 (20°C)', 'OO FBF (25°C)', 'OO FBF-2 (25°C)', 'OO FBF-1 (25°C)']
Couldn't find tag-173
Couldn't find bcl-1
Couldn't find vars-2
Couldn't find tofu-3
Couldn't find retr-1
Couldn't find prmt-3
Couldn't find W03A5.4
Couldn't find Y105E8A.24
Couldn't find hpo-17
Couldn't find pme-3
Couldn't find tag-170
Couldn't find Y10G11A.90
Couldn't find hpo-16
Ranked 1943 genes.
Check: 1943 gene names simplified to 

In [79]:
import random

from raw_david_output import *
importlib.reload(raw_david_output)

writerOut = pandas.ExcelWriter(table_dir + 'File S3 GO analysis.xlsx', engine='xlsxwriter')

def parse_david(text, sheet_name='sheetname'):
    _dicts = []
    text = text.split('\n')
    header = text[0].rstrip('\n').split('\t')
    print(header)
    for li in text[1:]:
        s = li.rstrip('\n').split('\t')
        _dicts.append(dict([(k, v) for k, v in zip(header, s)]))# if (k not in ['Genes'])]))
        if 'Genes' not in _dicts[-1]:
            print(_dicts[-1])
            #input()
        else:
            _dicts[-1]['Genes'] = len(_dicts[-1]['Genes'].split(' '))
        
        if ('Term' not in _dicts[-1]) or re.search('GO', _dicts[-1]['Term']) is None:
            _dicts = _dicts[:-1]
            
    df = pandas.DataFrame(_dicts)
    
    df['Benjamini'] = [float(x) for x in df.Benjamini]
    df = df[df['Benjamini']<0.01].copy()
    
    df = df[['Term', 'Benjamini', 'Fold Enrichment', '%']]
    
    df['Fold Enrichment'] = [float(x) for x in df['Fold Enrichment']]
    
    df['%'] = [float(x) for x in df['%']]
    df['% of RNAs with term'] = df['%']
    del df['%']
    
    df.to_excel(writerOut, sheet_name=sheet_name, index=False)
    
    worksheet = writerOut.sheets[sheet_name]
    workbook  = writerOut.book
    
    perc_format = workbook.add_format({'num_format': '.00'})
    
    worksheet.set_column('A:A', 50)
    worksheet.set_column('B:B', 20)
    worksheet.set_column('C:C', 20, perc_format)
    worksheet.set_column('D:D', 20, perc_format)
    print(df)

    return


parse_david(sp_david_top_500, sheet_name='SP FBF (25°C)')
parse_david(oo_david_top_500, sheet_name='OO FBF (25°C)')
parse_david(fbf1_low_temp_david, sheet_name='OO FBF-1 (20°C)')
parse_david(fbf2_low_temp_david, sheet_name='OO FBF-2 (20°C)')
parse_david(block_ii_david, sheet_name='Block II')
parse_david(block_iii_david, sheet_name='Block III')

writerOut.save()

['Category', 'Term', 'Count', '%', 'PValue', 'Genes', 'List Total', 'Pop Hits', 'Pop Total', 'Fold Enrichment', 'Bonferroni', 'Benjamini', 'FDR']
{'Category': ''}
                                                  Term     Benjamini  \
0                                GO:0007049~cell cycle  3.480244e-41   
1                        GO:0022402~cell cycle process  5.601027e-39   
2                          GO:0022403~cell cycle phase  5.943459e-34   
3    GO:0009792~embryonic development ending in bir...  1.565223e-33   
4                                   GO:0000279~M phase  1.486169e-33   
5                        GO:0051321~meiotic cell cycle  1.418189e-23   
6                             GO:0051301~cell division  1.282310e-23   
7                                   GO:0007126~meiosis  1.176704e-21   
8             GO:0051327~M phase of meiotic cell cycle  1.387779e-21   
9                               GO:0000910~cytokinesis  7.221311e-21   
10                   GO:0007059~chromosome se

In [65]:
??pandas.ExcelWriter